In [2]:
#PLATFORM GAME

import pygame as pg
import random 
from os import path

#import external file
%run settings.ipynb
%run sprites.ipynb

class Game:
    def __init__(self):
        pg.init()
        pg.mixer.init()
        self.screen = pg.display.set_mode((WIDTH, HEIGHT))
        pg.display.set_caption(TITLE)
        self.clock = pg.time.Clock()
        self.running = True
        #find the closest match to FONT_NAME
        self.font_name = pg.font.match_font(FONT_NAME)
        self.load_data()
        
    def load_data(self):
        #load high score
        self.dir = path.dirname(path.abspath("__file__"))
        img_dir = path.join(self.dir, "img")
        with open(path.join(self.dir, HS_FILE), "r") as f:
            try:
                self.highscore = int(f.read())
            except:
                self.highscore = 0
        self.spritesheet = Spritesheet(path.join(img_dir, SPRITESHEET))
        #Load sounds
        self.snd_dir = path.join(self.dir, "snd")
        
        self.jump_sound = pg.mixer.Sound(path.join(self.snd_dir, "jump.wav"))
        self.shoot_sound = pg.mixer.Sound(path.join(self.snd_dir, "ball_woosh.wav"))
        self.ammo_sound = pg.mixer.Sound(path.join(self.snd_dir, "yes.wav"))
        self.kiss_sound = pg.mixer.Sound(path.join(self.snd_dir, "kiss.wav"))
        self.powerup_sound = pg.mixer.Sound(path.join(self.snd_dir, "powerup.wav"))
        self.coin_sound =pg.mixer.Sound(path.join(self.snd_dir, "peach1.wav"))
        self.damage_sound = pg.mixer.Sound(path.join(self.snd_dir, "hurt.wav"))

    #start a new game
    def new(self):
        self.score = 0
        self.life = 100
        self.balls = 10
        
        self.all_sprites = pg.sprite.Group()
        self.platforms = pg.sprite.Group()
        self.grounds = pg.sprite.Group() 
        self.bullets = pg.sprite.Group()
        self.mobs = pg.sprite.Group()
        self.coins = pg.sprite.Group()
        self.powerups = pg.sprite.Group()
        self.ammos = pg.sprite.Group()
        self.runners = pg.sprite.Group()
        
        self.player = Player(self)
        self.all_sprites.add(self.player)  
        
        self.ground = Ground(0, HEIGHT - 40, WIDTH, 40)
        self.all_sprites.add(self.ground)
        self.grounds.add(self.ground)
        
        
        for plat in PLATFORM_LIST:
            #explode the list in components
            p = Platform(self, *plat)
            self.all_sprites.add(p)
            self.platforms.add(p)      
        
            c = Coins(self)
            self.all_sprites.add(c)
            self.coins.add(c)
        
        self.powerups_timer = 0
        self.mob_timer = 0
        self.runner_timer = 0
        
        pg.mixer.music.load(path.join(self.snd_dir, "theme.wav"))
        pg.mixer.music.set_volume(0.05)
        self.run()
    
    #handle the game loop
    def run(self):
        #keep on looping music
        pg.mixer.music.play(loops = -1)
        self.playing = True
        while self.playing:
            self.clock.tick(FPS)
            self.events()
            self.update()
            self.draw()
        #set fade out time 
        pg.mixer.music.fadeout(500)
        
    def update(self):
        self.all_sprites.update()
        
        #Spawn a powerup?
        #Spawn a Ball
        tres = 1000
        tres1 = 2000
        if self.score > 0:
            if round(self.score, -2) % tres ==0 or round(self.score, -2) % tres1 == 0:
                while len(self.powerups) < 1: 
                    powerup(self)
                while len(self.ammos) < 1:
                    Ammo(self)
            
        #Hit powerup
        powerup_hits = pg.sprite.spritecollide(self.player, self.powerups, True)
        if powerup_hits:
            self.powerup_sound.play()
            self.life += 25
                
        #Spawn a mob?
        now = pg.time.get_ticks()
        if now - self.mob_timer > 1650 + random.choice([-1000, -500, 500, 1000]):
            self.mob_timer = now
            Mob(self, (45, 60))
            
        #Hit mob
        mob_hits = pg.sprite.spritecollide(self.player, self.mobs, True)
        if mob_hits:
            self.damage_sound.play()
            self.life -= 25
            self.score -= 100
            if self.life <= 0:
                self.playing = False       
            
        #Runner generator
        tres = 4000
        tres1 = 5000
        if self.score > 0:
            if round(self.score, -2)%tres == 0 or round(self.score, -2)%tres1 == 0:
                while len(self.runners) < 1:
                    r = Runner(self)
                    self.runners.add(r)
                    self.all_sprites.add(r)

        #Hit runner
        runner_hits = pg.sprite.spritecollide(self.player, self.runners, True)
        if runner_hits:
            self.kiss_sound.play()
            self.life += 100
            self.score+= 1500
                
        #Hit bullets        
        bullet_hits = pg.sprite.groupcollide(self.bullets, self.mobs, True, True)
        for hit in bullet_hits:
            self.ammo_sound.play()
            self.score += 400
            
        #Hit coins
        if self.player.vel.x >0:
            hits = pg.sprite.spritecollide(self.player, self.coins, True)
            if hits:
                self.coin_sound.play()
                self.score += 200
            for coin in self.coins:
                if coin.rect.right <= WIDTH/4:
                    coin.kill()
                    
        #Hit ammo            
        if self.player.vel.x > 0:
            ammo_hits = pg.sprite.spritecollide(self.player, self.ammos, True)
            if ammo_hits:
                self.ammo_sound.play()
                self.balls += 5
            for ammo in self.ammos:
                if ammo.rect.right  < WIDTH/4:
                    ammo.kill()
                    
        #Player hits ground?
        if self.player.vel.y > 0:
            hits = pg.sprite.spritecollide(self.player, self.grounds, False)
            if hits:
                self.player.pos.y = hits[0].rect.top
                self.player.vel.y = 0          
                
        #Player hits platform?
        if self.player.vel.y > 0:
            hits = pg.sprite.spritecollide(self.player, self.platforms, False)
            if hits:
                self.player.pos.y = hits[0].rect.top
                self.player.vel.y = 0
                
        #Camera
        if self.player.rect.bottom >= HEIGHT - 35:
            self.player.pos.y -= abs(self.player.vel.y)
            self.ground.rect.y -= abs(self.player.vel.y)
            for plat in self.platforms:
                plat.rect.y -= abs(self.player.vel.y)
            
            #screen movement x    
        if self.player.rect.right >= WIDTH / 2:
            self.player.pos.x -= abs(self.player.vel.x)
            for plat in self.platforms:
                plat.rect.x -= abs(self.player.vel.x) 
                if plat.rect.right <= WIDTH / 4:
                    plat.kill()
         
        #GAME OVER --
        if self.player.rect.top > HEIGHT:
            self.playing = False
            
       #Plats generator     
        while len(self.platforms) < 4: 
            width = random.randrange(50, 400)
            p = Platform(self, WIDTH, random.randrange(300, HEIGHT*3/4),width, 10)
            self.platforms.add(p)
            self.all_sprites.add(p)

        #Coins generator
        while len(self.coins) < 1:
            c = Coins(self)
            self.coins.add(c)
            self.all_sprites.add(c)
            
                
    def events(self):
         for event in pg.event.get():
                if event.type == pg.QUIT:
                    if self.playing:
                        self.playing = False
                    self.running = False
                if event.type == pg.KEYDOWN:
                    if event.key == pg.K_SPACE:
                        self.player.jump()
                        self.jump_sound.play()
                    if event.key == pg.K_UP:
                        self.player.shoot()
                        self.shoot_sound.play()
                
    def draw(self):
        self.screen.fill(L_BLUE)
        self.all_sprites.draw(self.screen)
        self.draw_text("BALLS LEFT: "+str(self.balls), 22, WHITE, WIDTH *3/4, 30)
        self.draw_text("SCORE: "+str(self.score), 22, WHITE, WIDTH / 2, 30)
        self.draw_text("LIFE: " + str(self.life), 22, WHITE, WIDTH*1/4, 30 )
        pg.display.flip()
    
    def show_start_screen(self):
        #opening
        self.screen.fill(L_BLUE)
        self.draw_text(TITLE, 48, WHITE, WIDTH / 2, HEIGHT / 4)
        self.draw_text("controls information", 22, BLACK, WIDTH / 2, HEIGHT / 2)
        self.draw_text("Press any key to start!", 22, RED, WIDTH / 2, HEIGHT * 3/4)
        self.draw_text("High Score: " + str(self.highscore), 22, RED, WIDTH / 2, 15)
        pg.display.flip()
        self.wait_for_key()
    
    def show_go_screen(self):
        #game over
        if not self.running:
            return            
        self.screen.fill(BLACK)
        self.draw_text("GAME OVER", 48, WHITE, WIDTH / 2, HEIGHT / 4)
        self.draw_text("Score: "+str(self.score), 22, BLACK, WIDTH / 2, HEIGHT / 2)
        self.draw_text("play again", 22, RED, WIDTH / 2, HEIGHT * 3/4)     
        if self.score > self.highscore:
            self.highscore = self.score
            self.draw_text("NEW HIGH SCORE!", 22, WHITE, WIDTH / 2, HEIGHT /2 + 40)
            with open(path.join(self.dir, HS_FILE), "w") as f:
                f.write(str(self.score))
        else:
            self.draw_text("High score: " + str(self.highscore), 22, WHITE, WIDTH / 2, HEIGHT / 2 + 40)
                
        pg.display.flip()
        self.wait_for_key()
    
    def wait_for_key(self):
        waiting = True
        while waiting:
            self.clock.tick(60)
            for event in pg.event.get():
                #close
                if event.type == pg.QUIT:
                    waiting = False
                    self.running = False
                #Press any keys
                if event.type == pg.KEYUP:
                    waiting = False
    
    def draw_text(self, text, size, color, x, y):
        font = pg.font.Font(self.font_name, size)
        text_surface = font.render(text, True, color)
        text_rect = text_surface.get_rect()
        #positioning
        text_rect.midtop = (x, y)
        #show on screen
        self.screen.blit(text_surface, text_rect)

g  = Game()
g.show_start_screen()

#Start the loop
while g.running: 
    g.new()
    g.show_go_screen()

pg.quit()